In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import urllib
from zenrows import ZenRowsClient
from concurrent.futures import ThreadPoolExecutor
import psycopg2
from psycopg2 import sql
# !pip install sqlalchemy psycopg2-binary
from sqlalchemy import create_engine
from fuzzywuzzy import process

In [79]:
# response = client.get(url)
# response.raise_for_status()
# soup = BeautifulSoup(response.text, 'html.parser')

academic_years = {
    'AY24': [20241, 20233, 20232],
    'AY23': [20231, 20223, 20222],
    'AY22': [20221, 20213, 20212],
    'AY21': [20211, 20203, 20202],
    'AY20': [20201, 20193, 20192],
    'AY19': [20191, 20183, 20182],
    'AY18': [20181, 20173, 20172]
#     'AY17': [20171, 20163, 20162]
    # Add more academic years as needed
}
classes = ['PPD', 'PPDE', 'PLUS', 'RED', 'HMGT']

urls = []

for academic_year, terms in academic_years.items():
    for term in terms:
        for c in classes:
            url = f"https://classes.usc.edu/term-{term}/classes/{c}"
            urls.append(url)
#             response = client.get(url)
#             response.raise_for_status()
#             if response.status_code == 200:
#                 urls.append(url)
#                 print(f'{term}-{c}')
#             else:
#                 print(f'error with {term}-{c}')
#             soup = BeautifulSoup(response.text, 'html.parser')
            
print(urls)

['https://classes.usc.edu/term-20241/classes/PPD', 'https://classes.usc.edu/term-20241/classes/PPDE', 'https://classes.usc.edu/term-20241/classes/PLUS', 'https://classes.usc.edu/term-20241/classes/RED', 'https://classes.usc.edu/term-20241/classes/HMGT', 'https://classes.usc.edu/term-20233/classes/PPD', 'https://classes.usc.edu/term-20233/classes/PPDE', 'https://classes.usc.edu/term-20233/classes/PLUS', 'https://classes.usc.edu/term-20233/classes/RED', 'https://classes.usc.edu/term-20233/classes/HMGT', 'https://classes.usc.edu/term-20232/classes/PPD', 'https://classes.usc.edu/term-20232/classes/PPDE', 'https://classes.usc.edu/term-20232/classes/PLUS', 'https://classes.usc.edu/term-20232/classes/RED', 'https://classes.usc.edu/term-20232/classes/HMGT', 'https://classes.usc.edu/term-20231/classes/PPD', 'https://classes.usc.edu/term-20231/classes/PPDE', 'https://classes.usc.edu/term-20231/classes/PLUS', 'https://classes.usc.edu/term-20231/classes/RED', 'https://classes.usc.edu/term-20231/cl

In [ ]:
# now using threading

In [80]:
def parse_url(url):
    # Split the URL by '/' and filter out any empty strings
    parts = [part for part in url.split('/') if part]
    
    # Assuming the structure is always like this: [..., "term-XXXXX", "classes", "CLASS"]
    # Find the index of the part that starts with "term-"
    term_index = [i for i, part in enumerate(parts) if part.startswith("term-")][0]
    
    # Extract the term code and class abbreviation based on identified indexes
    term_code = parts[term_index].replace('term-', '')
    class_abbr = parts[term_index + 2]  # class abbreviation is two positions ahead of "term-XXXXX"

    return term_code, class_abbr


In [81]:
def process_url(url):
    try:
        term, class_abbr = parse_url(url)
        response = client.get(url)
        response.raise_for_status()  # Check if the request was successful
        soup = BeautifulSoup(response.text, 'html.parser')
        link_element = soup.find('div', class_='timestamp').find('a')
        if link_element:
            link_url = link_element.get('href')
            print(link_url)
            df = pd.read_csv(link_url)
            df.drop(columns=['Registration restrictions', 'Room', 'Waitlist'], inplace=True)
            # Add additional data to DataFrame
            df['term'] = term
            df['class_abbr'] = class_abbr
#             df['Department'] = c
            # Handle additional DataFrame columns as needed
            
            # This part is tricky in threading: returning or handling the combined DataFrame
            # Consider appending to a thread-safe collection or handling data merging after all threads complete
            return df
        else:
            print(f"No CSV link found at {url}")
    except Exception as e:
        print(f"Error processing {url}: {e}")

# You would need a thread-safe way to combine these DataFrames. This could be a global list,
# and then combine them after all threads complete.

In [82]:
combined_df = None  # Placeholder for your combined DataFrame

with ThreadPoolExecutor(max_workers=10) as executor:  # Adjust max_workers as needed
    futures = [executor.submit(process_url, url) for url in urls]
    for future in futures:
        df = future.result()
        if df is not None:
            if combined_df is None:
                combined_df = df
            else:
                combined_df = pd.concat([combined_df, df], ignore_index=True)


https://classes.usc.edu/term-20241/csv/red.csv
https://classes.usc.edu/term-20233/csv/hmgt.csv
https://classes.usc.edu/term-20241/csv/ppde.csv
https://classes.usc.edu/term-20233/csv/ppde.csv
https://classes.usc.edu/term-20233/csv/ppd.csv
https://classes.usc.edu/term-20233/csv/red.csv
https://classes.usc.edu/term-20241/csv/hmgt.csv
https://classes.usc.edu/term-20232/csv/ppd.csv
https://classes.usc.edu/term-20241/csv/ppd.csv
https://classes.usc.edu/term-20232/csv/plus.csv
https://classes.usc.edu/term-20232/csv/red.csv
https://classes.usc.edu/term-20231/csv/ppd.csv
https://classes.usc.edu/term-20231/csv/ppde.csv
https://classes.usc.edu/term-20231/csv/plus.csv
https://classes.usc.edu/term-20232/csv/hmgt.csv
https://classes.usc.edu/term-20231/csv/red.csv
https://classes.usc.edu/term-20232/csv/ppde.csv
https://classes.usc.edu/term-20223/csv/ppd.csv
https://classes.usc.edu/term-20223/csv/ppde.csv
https://classes.usc.edu/term-20233/csv/plus.csv
https://classes.usc.edu/term-20241/csv/plus.csv
h

In [84]:
combined_df
combined_df.to_csv("courses_raw.csv", index=False)

In [85]:
# print(combined_df.columns)
data = combined_df[['Course number', 'Course title', 'Units', 'Type', 'Registered', 'Instructor', 'term', 'class_abbr']]
print(data.columns)

Index(['Course number', 'Course title', 'Units', 'Type', 'Registered',
       'Instructor', 'term', 'class_abbr'],
      dtype='object')


In [ ]:
# cleaning time... drop 0 registered, convert units to int, check for multiple professors, 
# check for labs, forward fill everything, fuzzy match with excel faculty data, list of adjucts?

In [86]:
# forward fill course number, title, and units
data = data.copy()
data['Course number'].fillna(method='ffill', inplace=True)
data['Course title'].fillna(method='ffill', inplace=True)
data['Units'].fillna(method='ffill', inplace=True)
# quickly getting rid of 0 registered rows
data = data[data["Registered"] != 0]

In [15]:
# pd.unique(data['Units'])

array(['4.0 units', '2.0 units', '1.0-8.0 units, max 12',
       '1.0-8.0 units, max 8', '4.0', '1.0 unit', '3.0 units',
       '1.0-12.0 units', '1.0-3.0 units, max 3', '2.0-8.0 units, max 8',
       '1.0-12.0 units, max 18', '0.0 units', '2.0-4.0 units',
       '2.0-4.0 units, max 8', '2.0-4.0 units, max 12', '2.0',
       '4.0-12.0 units, max 12', '2.0 units, max 6',
       '1.0-4.0 units, max 12', '1.0-4.0 units', '3.0 units, max 6',
       '1.0 units, max 2', '6.0 units'], dtype=object)

In [87]:
def extract_units(value):
    if '-' not in value and 'max' not in value:
        return value.split()[0]
    else:
        return value
# Apply the function to the 'Units' column
data['Units'] = data['Units'].apply(extract_units)
# Display the modified DataFrame
# print(data[["Course number", "Units"]].head(60))
# get rid of courses with 0 units
data = data[data['Units'] != 0]

In [88]:
data = data[data['Units'] != '0.0']
# data["Units"].unique()

In [89]:
data.to_csv("courses_clean.csv", index=False)

In [ ]:
# need to add department column!

In [11]:
courses = pd.read_csv("courses_clean.csv")
scholar_links = pd.read_excel("scholar_links.xlsx")

In [13]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Function that takes an instructor name and returns the department using fuzzy matching
def get_department(instructor, scholar_df):
    # Check if the instructor name is a string, if not return 'Unknown'
    if not isinstance(instructor, str):
        return 'Unknown'
    
    # If there's an exact match, we return the department directly
    exact_match = scholar_df[scholar_df['Name'].str.lower() == instructor.lower()]['Department']
    if not exact_match.empty:
        return exact_match.iloc[0]

    # If there's no exact match, we perform a fuzzy match
    # We take the best match that has a score above a certain threshold (e.g., 80)
    best_match = process.extractOne(instructor, scholar_df['Name'], scorer=fuzz.token_sort_ratio)
    if best_match and best_match[1] >= 80:
        return scholar_df[scholar_df['Name'] == best_match[0]]['Department'].iloc[0]
    else:
        return 'Unknown'

# Normalize the names in the scholar_links DataFrame for better matching
# For the scope of this script, let's assume the normalization is lowercase only
scholar_links['Name'] = scholar_links['Name'].str.lower()

# Apply the function to each instructor in the courses DataFrame
courses['Department'] = courses['Instructor'].apply(lambda x: get_department(x, scholar_links))

# Save the result to a new CSV file
# updated_courses_file_path = '/mnt/data/updated_courses.csv'
courses.to_csv("courses_final.csv", index=False)

# Displaying the first few rows for verification
print(courses.head())


  Course number                                       Course title Units  \
0       PPD-225                            Solving Public Problems   4.0   
1      PPD-240g                      Citizenship and Public Ethics   4.0   
2      PPD-245g                                  The Urban Context   4.0   
3      PPD-245g                                  The Urban Context   4.0   
4       PPD-300  Social Justice Issues in Public Policy and Urb...   4.0   

      Type  Registered          Instructor   term class_abbr  \
0  Lecture        34.0           Danyao Li  20241        PPD   
1  Lecture        42.0        William Resh  20241        PPD   
2  Lecture        44.0        Dowell Myers  20241        PPD   
3  Lecture        39.0  Genevieve Giuliano  20241        PPD   
4  Lecture        45.0       Lavonna Lewis  20241        PPD   

                              Department  
0           Public Policy and Management  
1           Public Policy and Management  
2  Urban Planning and Spatial

In [ ]:
# we can filter by only matching faculty (unknown)

In [ ]:
# what to do about lecture / lab
# find where there are multiple teachers
# special topics/research

# if course ID the same, but the units is 1-12 and the course title is different, it is a special class
# RED-398	Shaping Cities Through Real Estate	2.0-4.0 units, max 8
# RED-398	Residential Properties	4
# RED-398	Retail Properties	4

In [ ]:
# databases!
# https://www.youtube.com/watch?v=v00O7ZXp-vU
# username btinsley, pw pricedataproject, dbname = 

In [68]:
# PGEND_POINT = 'database-2.cl6g6kie6d0b.us-east-1.rds.amazonaws.com'
# PGDATABASE_NAME = 'price_courses'
# PGUSER_NAME = 'btinsley'
# PGPASSWORD = 'pricedataproject'

# def connect():
#     conn_string = "host=" + PGEND_POINT +" port=" + "5432" +" dbname=" + PGDATABASE_NAME +" user=" + PGUSER_NAME +" password=" + PGPASSWORD
#     conn = psycopg2.connect(conn_string)
#     print("Connected")
    
#     cursor = conn.cursor()
    
#     return conn, cursor

In [67]:
# conn, cursor = connect()

In [51]:
# query_initial = sql.SQL("""
# CREATE TABLE IF NOT EXISTS Courses (
#     Course_number VARCHAR(255),
#     Course_title VARCHAR(255),
#     Units INT,
#     Type VARCHAR(100),
#     Section VARCHAR(100),
#     Session VARCHAR(100),
#     Time VARCHAR(100),
#     Days VARCHAR(100),
#     Seats INT,
#     Registered INT,
#     Instructor VARCHAR(255),
#     Term VARCHAR(100),
#     Class_abbr VARCHAR(100)
# );
# """)

# cur = conn.cursor()
# cur.execute(query_initial)
# cur.close()

In [52]:
# query_insert = sql.SQL("""
#     INSERT INTO Courses (
#         Course_number, 
#         Course_title, 
#         Units, 
#         Type, 
#         Section, 
#         Session, 
#         Time, 
#         Days, 
#         Seats, 
#         Registered, 
#         Instructor, 
#         Term, 
#         Class_abbr
#     ) VALUES (
#         {Course_number}, 
#         {Course_title}, 
#         {Units}, 
#         {Type}, 
#         {Section}, 
#         {Session}, 
#         {Time}, 
#         {Days}, 
#         {Seats}, 
#         {Registered}, 
#         {Instructor}, 
#         {Term}, 
#         {Class_abbr}
#     )
# """).format(
#     Course_number=sql.Literal('PPD-225'),
#     Course_title=sql.Literal('Solving Public Problems'),
#     Units=sql.Literal(4),
#     Type=sql.Literal('Lecture'),
#     Section=sql.Literal('51101R'),
#     Session=sql.Literal('1'),
#     Time=sql.Literal('4:00-5:50pm'),
#     Days=sql.Literal('Mon, Wed'),
#     Seats=sql.Literal(42),
#     Registered=sql.Literal(34),
#     Instructor=sql.Literal('Danyao Li'),
#     Term=sql.Literal('20241'),
#     Class_abbr=sql.Literal('PPD')
# )

# cur = conn.cursor()
# cur.execute(query_insert)
# conn.commit()
# cur.close()

In [69]:
# query_table = sql.SQL("""
# select * from Courses;
# """)

# cur = conn.cursor()
# cur.execute(query_table)
# print(cur.fetchone())

In [90]:
# Your database credentials
user = 'btinsley'
password = 'pricedataproject'
host = 'database-2.cl6g6kie6d0b.us-east-1.rds.amazonaws.com'
port = 5432
database = 'price_courses'

# Construct the engine connection string
conn_str = f'postgresql://{user}:{password}@{host}:{port}/{database}'

# Create the engine
engine = create_engine(conn_str, echo=True)  # Set echo=True to log all the SQL queries generated

# Test the connection
try:
    with engine.connect() as conn:
        print(conn.execute("SELECT 'Hello, World!'").fetchone())
except Exception as e:
    print(f"An error occurred: {e}")

2024-04-16 20:36:21,886 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-04-16 20:36:21,887 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-16 20:36:22,021 INFO sqlalchemy.engine.Engine select current_schema()
2024-04-16 20:36:22,021 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-16 20:36:22,155 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-04-16 20:36:22,156 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-16 20:36:22,287 INFO sqlalchemy.engine.Engine SELECT 'Hello, World!'
2024-04-16 20:36:22,288 INFO sqlalchemy.engine.Engine [raw sql] {}
('Hello, World!',)


In [91]:
drop_table_query = "DROP TABLE IF EXISTS Courses;"

try:
    with engine.connect() as connection:
        connection.execute(drop_table_query)
        print("Table 'Courses' dropped successfully.")
except Exception as e:
    print(f"An error occurred while dropping the table: {e}")

2024-04-16 20:36:33,902 INFO sqlalchemy.engine.Engine DROP TABLE IF EXISTS Courses;
2024-04-16 20:36:33,904 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-16 20:36:34,039 INFO sqlalchemy.engine.Engine COMMIT
Table 'Courses' dropped successfully.


In [92]:
create_table_query = """
CREATE TABLE IF NOT EXISTS Courses (
    Course_number VARCHAR(255),
    Course_title VARCHAR(255),
    Units VARCHAR(255),
    Type VARCHAR(100),
    Section VARCHAR(100),
    Session VARCHAR(100),
    Time VARCHAR(100),
    Days VARCHAR(100),
    Seats INT,
    Registered INT,
    Instructor VARCHAR(255),
    Term VARCHAR(100),
    Class_abbr VARCHAR(100)
);
"""

# Execute the create table command
try:
    with engine.connect() as connection:
        connection.execute(create_table_query)
        print("Table 'Courses' created successfully.")
except Exception as e:
    print(f"An error occurred while creating the table: {e}")


2024-04-16 20:36:38,036 INFO sqlalchemy.engine.Engine 
CREATE TABLE IF NOT EXISTS Courses (
    Course_number VARCHAR(255),
    Course_title VARCHAR(255),
    Units VARCHAR(255),
    Type VARCHAR(100),
    Section VARCHAR(100),
    Session VARCHAR(100),
    Time VARCHAR(100),
    Days VARCHAR(100),
    Seats INT,
    Registered INT,
    Instructor VARCHAR(255),
    Term VARCHAR(100),
    Class_abbr VARCHAR(100)
);

2024-04-16 20:36:38,037 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-16 20:36:38,195 INFO sqlalchemy.engine.Engine COMMIT
Table 'Courses' created successfully.


In [98]:
def process_url_and_insert_to_db(url, engine):
    try:
        term, class_abbr = parse_url(url)
        response = client.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        link_element = soup.find('div', class_='timestamp').find('a')
        if link_element:
            link_url = link_element.get('href')
            df = pd.read_csv(link_url)
            df.drop(columns=['Registration restrictions', 'Room', 'Waitlist'], inplace=True)
            df['term'] = term
            df['class_abbr'] = class_abbr

            # Ensure that numeric fields are converted properly
            df['Session'] = pd.to_numeric(df['Session'], errors='coerce')
            df['Seats'] = pd.to_numeric(df['Seats'], errors='coerce')
            df['Registered'] = pd.to_numeric(df['Registered'], errors='coerce')

            # Check and convert 'Days' column to ensure it doesn't cause issues
            df['Days'] = df['Days'].astype(str)  # Convert 'Days' to string to handle any unexpected non-string input

            # Insert into the database
            df.to_sql('Courses', con=engine, if_exists='append', index=False)
            print(f"Data inserted into the database for URL: {url}")
            
            return df  # Return the DataFrame for further processing
        else:
            print(f"No CSV link found at {url}")
            return pd.DataFrame()  # Return empty DataFrame in case of no data found
    except Exception as e:
        print(f"Error processing URL: {url}, Error: {e}")
        return pd.DataFrame()  # Return empty DataFrame in case of errors

def aggregate_data(urls):
    combined_df = pd.DataFrame()  # Initialize an empty DataFrame for aggregation
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(process_url_and_insert_to_db, url, engine): url for url in urls}
        for future in as_completed(futures):
            df = future.result()
            if not df.empty:
                combined_df = pd.concat([combined_df, df], ignore_index=True)
    
    return combined_df

# Use the function to process URLs and collect data
final_combined_df = aggregate_data(urls)
final_combined_df.to_csv("courses_raw.csv", index=False)

2024-04-16 20:46:08,807 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-16 20:46:08,811 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2024-04-16 20:46:08,812 INFO sqlalchemy.engine.Engine [cached since 367.7s ago] {'name': 'Courses'}
2024-04-16 20:46:08,860 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-16 20:46:08,862 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2024-04-16 20:46:08,863 INFO sqlalchemy.engine.Engine [cached since 367.8s ago] {'name': 'Courses'}
2024-04-16 20:46:08,921 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-16 20:46:08,923 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2024-

2024-04-16 20:46:09,020 INFO sqlalchemy.engine.Engine [cached since 367.2s ago] {'schema': 'public'}
2024-04-16 20:46:09,056 INFO sqlalchemy.engine.Engine INSERT INTO "Courses" ("Course number", "Course title", "Units", "Type", "Section", "Session", "Time", "Days", "Seats", "Registered", "Instructor", term, class_abbr) VALUES (%(Course number)s, %(Course title)s, %(Units)s, %(Type)s, %(Section)s, %(Session)s, %(Time)s, %(Days)s, %(Seats)s, %(Registered)s, %(Instructor)s, %(term)s, %(class_abbr)s)
2024-04-16 20:46:09,058 INFO sqlalchemy.engine.Engine [generated in 0.00170s] ({'Course number': 'PLUS-652', 'Course title': 'Place, Institutions, and Governance', 'Units': '4.0 units', 'Type': 'Lecture', 'Section': '51557D', 'Session': 1, 'Time': 'TBA', 'Days': 'nan', 'Seats': 18, 'Registered': 7, 'Instructor': 'William Leach', 'term': '20241', 'class_abbr': 'PLUS'}, {'Course number': 'PLUS-660', 'Course title': 'Economics of a Productive Development - A Public/Private Perspective', 'Units': 

Data inserted into the database for URL: https://classes.usc.edu/term-20241/classes/PLUS
2024-04-16 20:46:10,095 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-16 20:46:10,099 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2024-04-16 20:46:10,100 INFO sqlalchemy.engine.Engine [cached since 369s ago] {'name': 'Courses'}
2024-04-16 20:46:10,140 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-16 20:46:10,143 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2024-04-16 20:46:10,143 INFO sqlalchemy.engine.Engine [cached since 369.1s ago] {'name': 'Courses'}
2024-04-16 20:46:10,242 INFO sqlalchemy.engine.Engine INSERT INTO "Courses" ("Course number", "Course title", "Units", "Type", "Section", "Session", "Time", "Days", "Seats", "Register

2024-04-16 20:46:10,560 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-16 20:46:10,568 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2024-04-16 20:46:10,570 INFO sqlalchemy.engine.Engine [cached since 369.5s ago] {'name': 'Courses'}
2024-04-16 20:46:10,602 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-16 20:46:10,605 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2024-04-16 20:46:10,607 INFO sqlalchemy.engine.Engine [cached since 369.5s ago] {'name': 'Courses'}
2024-04-16 20:46:10,611 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-16 20:46:10,612 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2024-

2024-04-16 20:46:10,750 INFO sqlalchemy.engine.Engine [generated in 0.00248s] ({'Course number': 'PPDE-505', 'Course title': 'Professional Workshop in Public Administration', 'Units': '2.0 units', 'Type': 'Lecture', 'Section': '50907D', 'Session': 1, 'Time': '6:00-7:50pm', 'Days': 'Thursday', 'Seats': 20, 'Registered': 13, 'Instructor': 'Laura Wittcoff', 'term': '20241', 'class_abbr': 'PPDE'}, {'Course number': None, 'Course title': None, 'Units': None, 'Type': 'Lecture', 'Section': '50914D', 'Session': 61, 'Time': '6:00-7:50pm', 'Days': 'Tuesday', 'Seats': 20, 'Registered': 20, 'Instructor': 'Tara Blanc', 'term': '20241', 'class_abbr': 'PPDE'}, {'Course number': None, 'Course title': None, 'Units': None, 'Type': 'Lecture', 'Section': '50991D', 'Session': 530, 'Time': 'TBA', 'Days': 'nan', 'Seats': 28, 'Registered': 13, 'Instructor': 'Tara Blanc', 'term': '20241', 'class_abbr': 'PPDE'}, {'Course number': None, 'Course title': None, 'Units': None, 'Type': 'Lecture', 'Section': '51051D',

...


2024-04-16 20:46:47,476 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2024-04-16 20:46:47,476 INFO sqlalchemy.engine.Engine [cached since 406.4s ago] {'name': 'Courses'}
2024-04-16 20:46:47,484 INFO sqlalchemy.engine.Engine COMMIT
Error processing URL: https://classes.usc.edu/term-20181/classes/PPD, Error: (psycopg2.errors.InvalidTextRepresentation) invalid input syntax for type double precision: "Mon, Wed"
LINE 1: ... units', 'Lecture', '51100R', 1.0, '2:00-3:50pm', 'Mon, Wed'...
                                                             ^

[SQL: INSERT INTO "Courses" ("Course number", "Course title", "Units", "Type", "Section", "Session", "Time", "Days", "Seats", "Registered", "Instructor", term, class_abbr) VALUES (%(Course number)s, %(Course title)s, %(Units)s, %(Type)s, %(Section)s, %(Session)s, %(Time)s, %(Days)s, %(Seats)s, %(Registered)s, %(Instructor)s

2024-04-16 20:46:47,773 INFO sqlalchemy.engine.Engine INSERT INTO "Courses" ("Course number", "Course title", "Units", "Type", "Section", "Session", "Time", "Days", "Seats", "Registered", "Instructor", term, class_abbr) VALUES (%(Course number)s, %(Course title)s, %(Units)s, %(Type)s, %(Section)s, %(Session)s, %(Time)s, %(Days)s, %(Seats)s, %(Registered)s, %(Instructor)s, %(term)s, %(class_abbr)s)
2024-04-16 20:46:47,774 INFO sqlalchemy.engine.Engine [generated in 0.00079s] ({'Course number': 'HMGT-520', 'Course title': 'Leading People and Health Care Organizations', 'Units': '4.0 units', 'Type': 'Lecture', 'Section': '51710D', 'Session': 820, 'Time': 'TBA', 'Days': 'nan', 'Seats': 30, 'Registered': 22, 'Instructor': 'Gregory Chesley', 'term': '20181', 'class_abbr': 'HMGT'}, {'Course number': None, 'Course title': None, 'Units': None, 'Type': 'Lecture', 'Section': '51717D', 'Session': 820, 'Time': 'TBA', 'Days': 'nan', 'Seats': 30, 'Registered': 14, 'Instructor': 'Wayne Sass', 'term': 

2024-04-16 20:46:48,641 INFO sqlalchemy.engine.Engine INSERT INTO "Courses" ("Course number", "Course title", "Units", "Type", "Section", "Session", "Time", "Days", "Seats", "Registered", "Instructor", term, class_abbr) VALUES (%(Course number)s, %(Course title)s, %(Units)s, %(Type)s, %(Section)s, %(Session)s, %(Time)s, %(Days)s, %(Seats)s, %(Registered)s, %(Instructor)s, %(term)s, %(class_abbr)s)
2024-04-16 20:46:48,642 INFO sqlalchemy.engine.Engine [generated in 0.00109s] ({'Course number': 'PPD-301', 'Course title': 'PPD Practices: Internship Seminar', 'Units': '2.0 units', 'Type': 'Lecture', 'Section': '51393D', 'Session': 597.0, 'Time': 'TBA', 'Days': 'TBA', 'Seats': 40.0, 'Registered': 38.0, 'Instructor': 'Aggie Afarinesh', 'term': '20172', 'class_abbr': 'PPD'}, {'Course number': 'PPD-431', 'Course title': 'Undergraduate Policy, Planning, and Development Studio', 'Units': '4.0 units', 'Type': 'Lecture', 'Section': '51103D', 'Session': 588.0, 'Time': '2:00-5:00pm', 'Days': 'Tue, T

2024-04-16 20:46:49,079 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-16 20:46:49,081 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2024-04-16 20:46:49,082 INFO sqlalchemy.engine.Engine [cached since 408s ago] {'name': 'Courses'}
2024-04-16 20:46:49,144 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-16 20:46:49,145 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2024-04-16 20:46:49,146 INFO sqlalchemy.engine.Engine [cached since 408.1s ago] {'name': 'Courses'}
2024-04-16 20:46:49,219 INFO sqlalchemy.engine.Engine INSERT INTO "Courses" ("Course number", "Course title", "Units", "Type", "Section", "Session", "Time", "Days", "Seats", "Registered", "Instructor", term, class_abbr) VALUES (%(Course number)s, %(Course title)s, %(Units

Error processing URL: https://classes.usc.edu/term-20172/classes/HMGT, Error: (psycopg2.errors.InvalidTextRepresentation) invalid input syntax for type double precision: "TBA"
LINE 1: ...t', '2.0 units', 'Lecture', '51722D', 820, 'TBA', 'TBA', 27,...
                                                             ^

[SQL: INSERT INTO "Courses" ("Course number", "Course title", "Units", "Type", "Section", "Session", "Time", "Days", "Seats", "Registered", "Instructor", term, class_abbr) VALUES (%(Course number)s, %(Course title)s, %(Units)s, %(Type)s, %(Section)s, %(Session)s, %(Time)s, %(Days)s, %(Seats)s, %(Registered)s, %(Instructor)s, %(term)s, %(class_abbr)s)]
[parameters: ({'Course number': 'HMGT-512', 'Course title': 'Information Technology and Patient Engagement', 'Units': '2.0 units', 'Type': 'Lecture', 'Section': '51722D', 'Session': 820, 'Time': 'TBA', 'Days': 'TBA', 'Seats': 27, 'Registered': 20, 'Instructor': 'Wayne Sass', 'term': '20172', 'class_abbr': 'HMGT'}, {'Course number